# 3번과제

In [5]:
#  과제 3: 생일 데이터를 교재 heap.py 코드로 힙에 저장하고, 생일이 느린 순서로 10명 출력

import pandas as pd
from datetime import datetime

# 1. 생일 데이터 불러오기
df = pd.read_csv("birthday.csv")

# 2. 생년월일 열 정제 및 datetime 형식으로 변환
df["생년월일8자리(예.20040101)"] = df["생년월일8자리(예.20040101)"].astype(str).str.replace(".0", "", regex=False)
df["생일"] = pd.to_datetime(df["생년월일8자리(예.20040101)"], format="%Y%m%d", errors='coerce')

# 3. 교재 heap.py 코드 복붙
class Heap:
    def __init__(self, *args):
        if len(args) != 0:
            self.__A = args[0]
        else:
            self.__A = []

    def insert(self, x):
        self.__A.append(x)
        self.__percolateUp(len(self.__A) - 1)

    def __percolateUp(self, i: int):
        parent = (i - 1) // 2
        if i > 0 and self.__A[i] > self.__A[parent]:
            self.__A[i], self.__A[parent] = self.__A[parent], self.__A[i]
            self.__percolateUp(parent)

    def deleteMax(self):
        if not self.isEmpty():
            max_item = self.__A[0]
            last_item = self.__A.pop()
            if not self.isEmpty():
                self.__A[0] = last_item
                self.__percolateDown(0)
            return max_item
        else:
            return None

    def __percolateDown(self, i: int):
        child = 2 * i + 1
        right = 2 * i + 2
        if child <= len(self.__A) - 1:
            if right <= len(self.__A) - 1 and self.__A[child] < self.__A[right]:
                child = right
            if self.__A[i] < self.__A[child]:
                self.__A[i], self.__A[child] = self.__A[child], self.__A[i]
                self.__percolateDown(child)

    def max(self):
        return self.__A[0]

    def buildHeap(self):
        for i in range((len(self.__A) - 2) // 2, -1, -1):
            self.__percolateDown(i)

    def isEmpty(self) -> bool:
        return len(self.__A) == 0

    def clear(self):
        self.__A = []

    def size(self) -> int:
        return len(self.__A)

# 4. 힙 생성 후 생일 데이터를 timestamp 기준으로 저장
heap = Heap()
for _, row in df.iterrows():
    if pd.notnull(row["생일"]):
        heap.insert((row["생일"].timestamp(), row["이름"]))


top_10 = []
for _ in range(10):
    if not heap.isEmpty():
        timestamp, name = heap.deleteMax()
        birthday = datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d")
        top_10.append((name, birthday))

# 출력 결과 보여주기

for i, (name, birthday) in enumerate(top_10, 1):
    print(f"{i}. {name} - {birthday}")



1. 신수민 - 2005-12-30
2. 이서영 - 2005-12-25
3. 강민주 - 2005-12-14
4. 김민경 - 2005-12-02
5. 이서영 - 2005-11-12
6. 배시은 - 2005-11-02
7. 김여원 - 2005-10-31
8. 이서진 - 2005-10-28
9. 서홍빈 - 2005-10-24
10. 김예빈 - 2005-10-19


#4번 과제

In [6]:
# 📌 과제 4: 이름 + 생일 기준으로 정확하게 조원만 출력 (동명이인 필터링 포함)

import pandas as pd
from datetime import datetime

# 1. CSV 불러오기
df = pd.read_csv("birthday.csv")
df["생년월일8자리(예.20040101)"] = df["생년월일8자리(예.20040101)"].astype(str).str.replace(".0", "", regex=False)
df["생일"] = pd.to_datetime(df["생년월일8자리(예.20040101)"], format="%Y%m%d", errors='coerce')

# 2. 조원 이름 + 생일 쌍 (확실한 필터링용)
group2_name_birth = [
    ("선예영", "2003-01-22"),
    ("황채원", "2004-08-13"),
    ("나현주", "2004-12-13"),
    ("윤서현", "2004-03-15"),
    ("이승주", "2004-10-05"),
    ("조승연", "2005-01-19"),
    ("남수연", "2003-07-11"),
    ("김민선", "2003-05-04"),
    ("김하린", "2003-02-12"),
    ("이서진", "2005-10-28")  # 이 생일에 해당하는 이서진만 포함
]

# 3. 연결 리스트 클래스 정의
class BidirectNode:
    def __init__(self, item, prev, next):
        self.item = item
        self.prev = prev
        self.next = next

class CircularDoublyLinkedList:
    def __init__(self):
        self.__head = BidirectNode("dummy", None, None)
        self.__head.prev = self.__head
        self.__head.next = self.__head
        self.__numItems = 0

    def append(self, newItem) -> None:
        prev = self.__head.prev
        newNode = BidirectNode(newItem, prev, self.__head)
        prev.next = newNode
        self.__head.prev = newNode
        self.__numItems += 1

    def __iter__(self):
        return CircularDoublyLinkedListIterator(self)

    def getNode(self, i: int) -> BidirectNode:
        curr = self.__head
        for index in range(i + 1):
            curr = curr.next
        return curr

class CircularDoublyLinkedListIterator:
    def __init__(self, alist):
        self.__head = alist.getNode(-1)
        self.iterPosition = self.__head.next

    def __next__(self):
        if self.iterPosition == self.__head:
            raise StopIteration
        else:
            item = self.iterPosition.item
            self.iterPosition = self.iterPosition.next
            return item

    def __iter__(self):
        return self

# 4. 리스트에 (이름, 생일) 저장
cdll = CircularDoublyLinkedList()
for _, row in df.iterrows():
    if pd.notnull(row["생일"]):
        name = row["이름"]
        birth = row["생일"].strftime("%Y-%m-%d")
        cdll.append((name, birth))

# 5. 정확한 이름+생일 쌍에 포함되는 경우만 출력

for name, birth in cdll:
    if (name, birth) in group2_name_birth:
        print(f"{name} - {birth}")


김하린 - 2003-02-12
선예영 - 2003-01-22
이서진 - 2005-10-28
이승주 - 2004-10-05
조승연 - 2005-01-19


#5 교재 8장 연습 문제

###1)
임의의 최대 힙에서 더 얕은 곳에 있는 원소가 더 깊은 곳에 있는 원소보다 더 작은 값을 가질 수 있다.
예를 들어보면, heap[1] = 7, heap[2] = 10, heap[5] = 9인 힙이 있다. heap[1]과 heap[2]는 깊이가 같고, heap[5]는 heap[1]보다 깊이가 깊지만 값은 작다. heap[5]는 heap[2]의 자식 노드로, heap[2] > heap[5]이기 때문에 힙의 조건을 만족한다.


###2)
최대 힙의 마지막 원소가 항상 가장 작은 값은 아니다.
heap[0] = 10, heap[1] = 7, heap[2] = 9이고 이 두 노드가 말단 노드인 힙이 있다고 한다. 이 힙은 heap[0] > heap[1], heap[0] > heap[2]가 되어 힙의 조건을 만족하며, heap[n-2] < heap[n-1]을 만족하는 힙이다.

###3)
길이가 n인 임의의 리스트를 힙으로 만들 때, 마지막 요소의 인덱스는 n-1이다. 따라서 인덱스가 ((n-1)-1)//2 즉 (n-2)//2부터 0까지의 인덱스를 갖는 요소를 대상으로 스며내리기를 진행한다.
따라서 루트의 자격으로 스며내리기를 하지 않고 그냥 넘어가는 원소 수는 정확하게 n-1-(n-2)//2개이다.

###4)
길이가 n인 리스트를 대상으로 하는 스며내리기 알고리즘에서 최악의 경우 Θ(logn)의 시간이 소요된다. 이진 탐색 트리의 깊이에 따라 층을 나누면 의 개수는 약 (logn + 1)개가 된다. 스며내리기의 대상이 되는 리스트의 인덱스가 0인 요소가 이 리스트의 가장 작은 값이라면 최악의 경우가 되어, 스며내리기를 logn번 진행해야 한다.
최선의 경우에는 Θ(1)의 시간이 소요된다. 스며내리기의 대상이 되는 리스트의 인덱스가 0인 요소가 이 리스트의 가장 큰 값이라면 인덱스가 1, 2인 요소보다 크기 때문에 스며내리기는 진행되지 않고 멈춘다.

###5)
이 문제에서 말하는 마지막 원소가 단순히 리스트의 마지막 원소라면 매우 간단히 원소를 삭제할 수 있다.

하지만 여기서 말하는 마지막 원소가 우선순위의 마지막이 되는 원소라면 힙의 마지막 원소를 삭제하는 작업은 그렇게 간단하지는 않다. 연습문제 2번에서 확인한 바와 같이 마지막 원소가 항상 가장 작은 값을 가지지도 않고, 연습문제 1번에서 확인한 바와 같이 깊이가 깊은 원소가 항상 깊이가 얕은 원소보다 작은 값을 가지지도 않는다.
하지만 힙의 특성상 자식 노드는 부모 노드보다 작은 값을 가지기 때문에, 깊이가 2인 모든 서브 트리들의 원소들만 비교하면 가장 마지막 원소를 어렵지 않게 찾을 수 있다. 겨우 깊이가 2인 서브 트리이기 때문에 원소의 삭제 작업도 간단하다.

###6)
위쪽부터 시작해서 스며오르기를 반복하여 build_heap() 알고리즘을 구현한다고 해보자. index가 0인 요소에는 이미 맨 위의 요소이므로 스며오르기를 할 수 없다. 따라서 index가 1인 요소부터 index가 n-1(마지막)인 요소까지 진행한다. 처음 스며오르기를 진행할 땐 대상 노드의 깊이가 얕기 때문에 비교와 교체 작업이 많지 않지만, index가 커질수록 대상 노드의 깊이가 깊어져 스며오르기를 logn번 진행해야 할 수도 있다.
n/2 * logn + n/4 * log(n-1) + n/8 * log(n-2) + ... + n/(2**(n-1)) * log2 즉 Θ(nlogn)의 시간이 소요된다. 이 방법은 Θ(n)의 시간이 소요되는 스며내리기를 이용한 build_heap() 알고리즘보다 비효율적이다.

###7)
임의의 원소의 값이 증가했다면, 그 원소에 해당하는 노드를 제거하는 delete_max() 작업을 실행한다. 이 작업에 소요되는 시간은 O(logn)이다. 그리고 증가한 원소의 값을 갖는 노드를 힙에 추가한다. 노드를 추가하는 작업 insert()는 O(logn)의 시간이 든다.
이 두 작업을 통해 힙을 O(logn) 시간만에 변화를 반영하여 힙을 수선할 수 있다

#6번 LeetCode 703.Kth Largest Element in Stream

In [ ]:
import heapq

class KthLargest:

    def __init__(self, k: int, nums: list[int]):
        self.k = k
        self.heap = []
        for num in nums:
            self.add(num)  # 초기 값도 힙에 넣기

    def add(self, val: int) -> int:
        heapq.heappush(self.heap, val)  # 값 추가
        if len(self.heap) > self.k:     # 힙 크기 초과 시 가장 작은 값 제거
            heapq.heappop(self.heap)
        return self.heap[0]             # k번째 큰 값은 최소 힙의 루트
